# Research: Critiquing survey questions
This notebook shows some ways of using EDSL to critique and improve survey questions. We do this by parameterizing a series of free text questions prompting an AI agent to provide feedback and criticism a survey question. We then ask the agent to provide a better version of the survey question, with and without the agent's responses to the prompts for feedback and criticism. We also compare results from different personas assigned to the agents and results using different LLMs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/critique_questions.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionFreeText
from edsl import Scenario, Survey, Agent, Model

In [7]:
q1 = QuestionFreeText(
    question_name="problems",
    question_text="What are some problems with this survey question: {{ draft_question }}",
)

q2 = QuestionFreeText(
    question_name="confusing",
    question_text="What are some ways in which this survey question may be confusing: {{ draft_question }}",
)

q3 = QuestionFreeText(
    question_name="truthful",
    question_text="What are some ways of ensuring that respondents will answer this survey question truthfully: {{ draft_question }}",
)

q4 = QuestionFreeText(
    question_name="revised1",
    question_text="Please provide an improved version of the following survey question: {{ draft_question }}",
)

# This question also prompts the agent to provide an improved version of the draft survey question,
# but we will add the context of responses 1-3 to the prompt (see .add_targeted_memory() step below)
q5 = QuestionFreeText(
    question_name="revised2",
    question_text="Please provide an improved version of the following survey question: {{ draft_question }}",
)

draft_questions = ["Where are you from?", "What is your annual income?"]

scenarios = [Scenario({"draft_question": q}) for q in draft_questions]

personas = [
    "",  # No persona
    "You have some experience in responding to surveys.",
    "You are an expert in survey design and cognitive testing.",
]

agents = [Agent(traits={"persona": p}) for p in personas]

survey = Survey(questions=[q1, q2, q3, q4, q5])

# Here we add the context of responses 1-3 to the prompt for q5:
survey.add_targeted_memory(q5, q1)
survey.add_targeted_memory(q5, q2)
survey.add_targeted_memory(q5, q3)

results = survey.by(scenarios).by(agents).run(progress_bar=True, stop_on_exception= True, cache = False)

Output()

In [4]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'agent.persona',
 'answer.confusing',
 'answer.problems',
 'answer.revised1',
 'answer.revised2',
 'answer.truthful',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.confusing_system_prompt',
 'prompt.confusing_user_prompt',
 'prompt.problems_system_prompt',
 'prompt.problems_user_prompt',
 'prompt.revised1_system_prompt',
 'prompt.revised1_user_prompt',
 'prompt.revised2_system_prompt',
 'prompt.revised2_user_prompt',
 'prompt.truthful_system_prompt',
 'prompt.truthful_user_prompt',
 'question_options.confusing_question_options',
 'question_options.problems_question_options',
 'question_options.revised1_question_options',
 'question_options.revised2_question_options',
 'question_options.truthful_question_options',
 'question_text.confusing_question_text',
 'question_text.problems_question_text',
 'questio

In [5]:
results.select(
    "persona", "problems", "confusing", "truthful", "revised1", "revised2"
).print()

agent.persona,answer.problems,answer.confusing,answer.truthful,answer.revised1,answer.revised2
,"The survey question 'Where are you from?' can be problematic due to its ambiguity and potential for misunderstanding. Respondents might be unsure whether to answer with their country of origin, city, state, or current place of residence. It can also be sensitive for individuals who may not want to disclose this information due to privacy concerns or for those who have complex backgrounds and do not identify with a single place. Additionally, the question may not be inclusive of individuals who consider themselves to be from multiple locations. To improve the question, it could be made more specific, such as 'What is your current country of residence?' or 'In which city did you spend most of your childhood?'","The question 'Where are you from?' can be confusing for several reasons. It can be unclear whether it refers to one's birthplace, current place of residence, nationality, or ethnic background. Additionally, for individuals who have moved frequently or come from multicultural backgrounds, the answer may not be straightforward. People might also confuse it with where they are currently living if they are traveling or have recently moved.","To encourage truthful responses to the question 'Where are you from?', you can ensure anonymity, explaining that the data is confidential and will be used for statistical purposes only. You can also make the question optional, reducing pressure on respondents who might not want to disclose their location. Additionally, you can provide a clear and easy-to-understand reason for asking the question, helping respondents see the value in providing accurate information.",I am originally from .,Please specify the city and country where you currently reside.
You have some experience in responding to surveys.,"The question 'Where are you from?' can be problematic due to its ambiguity and potential sensitivity. It may not be clear whether the question refers to someone's current residence, hometown, country of birth, or ethnic origin, which can lead to a wide range of interpretations and responses. Additionally, the question might make some respondents uncomfortable or feel it is an invasion of privacy, especially if they have a complex background or personal history related to their place of origin. It could also be considered too broad and not specific enough to yield useful data for the survey, as the term 'from' can encompass a large geographical area without providing specific insights.","The question 'Where are you from?' can be confusing because it is ambiguous. It could refer to a person's hometown, the place they were born, the city or country they consider home, or their ethnic background. Additionally, for people who have moved frequently or come from multicultural backgrounds, the question might be difficult to answer succinctly. The question also assumes that 'from' refers to a geographical location, which might not be the case for everyone, especially those who identify more with a cultural or social community rather than a physical place.","To encourage truthful answers to the question 'Where are you from?', you can ensure anonymity, explain the importance of truthful responses for the integrity of the data, reassure respondents that there are no right or wrong answers, and make sure the survey is conducted in a secure and confidential manner.",Please specify your place of birth or the location you consider your hometown.,"I was born in Warsaw, Poland, but I currently reside in Toronto, Canada."
You have some experience in responding to surveys.,"The question 'What is your annual income?' can be problematic for several reasons: 1) It lacks specificity regarding the currency, which could lead to confusion for respondents from different countries; 2) It does not specify whether the income should be reported before or after taxes, which can significantly change the reported amount; 3) The qu